In [24]:
# https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        # 1 input channel, 6 output channels, 5x5 square convalution
        self.conv1 = nn.Conv2d(1, 6, 5)
        
        # 6 input channel, 16 output channels, 5x5 square convalution
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        return self.fc3(x)
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features

In [7]:
net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
params = list(net.parameters())
print(len(params))
print(params[2].size())

10
torch.Size([16, 6, 5, 5])


In [19]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1179, -0.1291, -0.0111, -0.0381, -0.0790,  0.0119,  0.0436,  0.0518,
         -0.0975,  0.0984]], grad_fn=<AddmmBackward>)


In [21]:
# compute the loss
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)

tensor(0.4002, grad_fn=<MseLossBackward>)


In [22]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])

In [23]:
# backprop
net.zero_grad()

print(net.conv1.bias.grad)

loss.backward()

print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([ 0.0070,  0.0085, -0.0004,  0.0032, -0.0065, -0.0011])


In [26]:
# update weights

optimizer = optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad()
out = net(input)
loss = criterion(out, target)
loss.backward()
optimizer.step()